In [ ]:
from imgaug import augmenters as iaa
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import os

In [ ]:
read_image = lambda path: cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)

In [ ]:
# Load sample datasets
normal_images = list(map(read_image, ['muestra/' + path for path in os.listdir('muestra/')]))

limit_images = list(map(read_image, ['sample/' + path for path in os.listdir('sample/')]))

# Data augmentation
## Brightness, rotation and HUE offset

In [ ]:
# Original data augmentation

# Data augmentation parameters
brightness_range = (-40, +50)

#contrast_range = (0.65, 1.6)

rotation_range = (-5, 5)

hue_offset_range = (-10, 10)

## Visualize limit values of parameters

In [ ]:
# This functions permits the user see every transformation applied using its limit parameters
def visualize_limits(images, augs, figsize=(22,22)):
    num_elements = len(images)
    num_cols = len(augs) * 2 + 1 if len(augs[0]) == 3 else len(augs) + 1

    plt.subplots(num_elements,num_cols, figsize=figsize)
    
    count = 1
    for img in images:

        plt.subplot(num_elements,num_cols,count)
        plt.imshow(img)
        plt.xticks([]);plt.yticks([])
        plt.title('Original')
        count += 1
        
        # Get every transformation and apply it
        for e in augs:
            if len(e) == 3:
                aug_1, aug_2, name = e

                # Low limit
                plt.subplot(num_elements,num_cols, count)
                plt.imshow(aug_1.augment_image(img))
                plt.xticks([]);plt.yticks([])
                plt.title(name + ' Min')
                count += 1
                
                # High limit
                plt.subplot(num_elements,num_cols, count)
                plt.imshow(aug_2.augment_image(img))
                plt.xticks([]);plt.yticks([])
                plt.title(name + ' Max')
                count += 1
            else:
                aug_1, name = e

                plt.subplot(num_elements,num_cols, count)
                plt.imshow(aug_1.augment_image(img))
                plt.xticks([]);plt.yticks([])
                plt.title(name)
                count += 1

    plt.show()

In [ ]:
# Visualize data augmentations parameters
# These are all augmentation operations with their extreme values
augs = [
    #[iaa.Fliplr(0.5)], # We ignore left-right flip, we all know its output
    [iaa.Affine(rotate=rotation_range[0]), iaa.Affine(rotate=rotation_range[1]), 'Rotation'],
    [iaa.AddToBrightness(add=brightness_range[0], to_colorspace='YCrCb'), iaa.AddToBrightness(add=brightness_range[1], to_colorspace='YCrCb'), 'Brightness'],
    #[iaa.LinearContrast(alpha=contrast_range[0]), iaa.LinearContrast(alpha=contrast_range[1]), 'Contrast'],
    [iaa.AddToHue(value=hue_offset_range[0]), iaa.AddToHue(value=hue_offset_range[1]), 'HUE offset']
]

# Visualize limit values on normal images
visualize_limits(normal_images, augs)

# Visualize limit values on extreme images
visualize_limits(limit_images, augs)

## Random Noise on images

In [ ]:
uniform_noise_range = (-4, 4)
gaussian_noise_desv = 6
laplacian_noise = 6
poisson_noise = 6

augs = [
    [iaa.AddElementwise(uniform_noise_range, per_channel=True), 'Uniform'],
    [iaa.AdditiveGaussianNoise(scale=(0, gaussian_noise_desv), per_channel=True), 'Gaussian'],
    [iaa.AdditiveLaplaceNoise(scale=(0, laplacian_noise), per_channel=True), 'Laplacian'],
    [iaa.AdditivePoissonNoise((0, poisson_noise), per_channel=True), 'Poisson']
]

# See effects on all images
# visualize_limits(normal_images, augs, figsize=(26,26))
# visualize_limits(limit_images, augs, figsize=(26,26))

# See effects on one single image, with higher resolution
img = normal_images[0]

plt.subplots(len(augs), 2, figsize=(20,40))
i = 1
for aug, name in augs:
    plt.subplot(len(augs), 2, i)
    plt.imshow(img)
    plt.title('Original')
    i += 1

    plt.subplot(len(augs), 2, i)
    plt.imshow(aug.augment_image(img))
    plt.title(name)
    i += 1
plt.show()

## Applying fog on images

In [ ]:
fog_augmenter = iaa.CloudLayer(intensity_mean=150, 
                               intensity_freq_exponent=-1.5,
                               intensity_coarse_scale=1,
                               alpha_min=0.1,
                               alpha_multiplier=0.3,
                               alpha_size_px_max=50,
                               alpha_freq_exponent=-2,
                               sparsity=1.01,
                               density_multiplier=0.5)

augs = [[fog_augmenter, 'Fog']]

visualize_limits(normal_images, augs, figsize=(30,80))
visualize_limits(limit_images, augs, figsize=(20,44))

# Fog won't be applied

# Define custom non-linear contrast functions
#### Using linear contrast function can give good results, but using non-linear functions may return better images
#### These functions will have two ranges: positive range and negative range

In [ ]:
# Contrast functions
def gamma(img: np.ndarray, exponent: float):
    # Positive exponent: increase contrast on lighter areas of the image
    # Negative exponent: increase contrast on darker areas of the image
    img = img.astype(np.float32)
    return img ** (1+exponent) / 1 ** exponent

def sigmoid_limits(img: np.ndarray, alfa: float):
    img = img.astype(np.float32)
    img_ = (1/2) * (1 + (1 / np.tan((np.pi * alfa) / 2)) * np.tan(alfa * np.pi * (img/1 - 1/2)))
    return img_

def sigmoid_limits_simmetric(img: np.ndarray, alfa: float):
    # Positive alfa: increase contrast on extreme values of the image (darker and lighter)
    # Negative alfa: increase contrast on middle values of the image
    F = sigmoid_limits(img, alfa)
    
    if alfa > 0:
        return F
    else:
        I = img.astype(np.float32)
        F = F.astype(np.float32)
        F = np.clip(2*I - F, 0, 1)
 
        return F

In [ ]:
# See how these functions would work
# View functions

# These are their ranges. Negative ranges will be symmetric
gamma_range = (0.35, 0.5)

sigmoid_range = (0.52, 0.68)

def view_function(function, range, x_values):
    # Low value of range
    y_range_0 = function(x_values, range[0])
    plt.plot(x_values, y_range_0, label=str(range[0]))

    # High value of range
    y_range_0 = function(x_values, range[1])
    plt.plot(x_values, y_range_0, label=str(range[1]))

    plt.legend(loc='best')

    plt.title(function.__name__)

functions = [gamma, sigmoid_limits_simmetric]
ranges = [gamma_range, sigmoid_range]

plt.subplots(1, len(functions) * 2, figsize=(26,4))

# X values mean original pixel values: [0, 1] (Normalized)
x = np.linspace(0, 1, 400)

count = 1
for f, rng in zip(functions, ranges):
    plt.subplot(1, len(functions) * 2, count)
    view_function(f, rng, x)
    count += 1

    plt.subplot(1, len(functions) * 2, count)
    view_function(f, (-rng[1], -rng[0]), x)
    count += 1
plt.show()

These functions will be applied on HSI and RGB channels, so we need RGB to HSI and HSI to RGB functions

In [ ]:
# RGB to HSI and HSI to RGB functions

# These functions were taken from this site
# https://stackoverflow.com/questions/52939362/trouble-displaying-image-hsi-converted-to-rgb-python

def rgb_to_hsi(img):
    zmax = 255 # max value
    # values in [0,1]
    R = np.divide(img[:,:,0],zmax,dtype=np.float)
    G = np.divide(img[:,:,1],zmax,dtype=np.float)
    B = np.divide(img[:,:,2],zmax,dtype=np.float)

    # Hue, when R=G=B -> H=90
    a = (0.5)*np.add(np.subtract(R,G), np.subtract(R,B)) # (1/2)*[(R-G)+(R-B)]
    b = np.sqrt(np.add(np.power(np.subtract(R,G), 2) , np.multiply(np.subtract(R,B),np.subtract(G,B))))
    tetha = np.arccos( np.divide(a, b, out=np.zeros_like(a), where=b!=0) ) # when b = 0, division returns 0, so then tetha = 90
    H = (180/np.pi)*tetha # convert rad to degree
    H[B>G]=360-H[B>G]

    # saturation = 1 - 3*[min(R,G,B)]/(R+G+B), when R=G=B -> S=0
    a = 3*np.minimum(np.minimum(R,G),B) # 3*min(R,G,B)
    b = np.add(np.add(R,G),B) # (R+G+B)
    S = np.subtract(1, np.divide(a,b,out=np.ones_like(a),where=b!=0))

    # intensity = (1/3)*[R+G+B]
    I = (1/3)*np.add(np.add(R,G),B)

    # return np.dstack((H, zmax*S, np.round(zmax*I))) # values between [0,360], [0,255] e [0,255]

    return np.dstack((H / 360.0, S, I)) # values between [0, 1], [0, 1] and [0, 1]

def hsi_to_rgb(img):

    def f1(I,S): # I(1-S)
        return np.multiply(I, np.subtract(1,S))
    def f2(I,S,H): # I[1+(ScosH/cos(60-H))]
        r = np.pi/180
        a = np.multiply(S, np.cos(r*H)) # ScosH
        b = np.cos(r*np.subtract(60,H)) # cos(60-H)
        return np.multiply(I, np.add(1, np.divide(a,b)) )
    def f3(I,C1,C2): # 3I-(C1+C2)
        return np.subtract(3*I, np.add(C1,C2))

    zmax = 255 # max value
    # values between[0,1], [0,1] and [0,1]
    H = img[:,:,0] * 360
    # S = np.divide(img[:,:,1],zmax,dtype=np.float)
    # I = np.divide(img[:,:,2],zmax,dtype=np.float)
    S = img[:,:,1]
    I = img[:,:,2]

    R,G,B = np.ones(H.shape),np.ones(H.shape),np.ones(H.shape) # values will be between [0,1]
    # for 0 <= H < 120
    B[(0<=H)&(H<120)] = f1(I[(0<=H)&(H<120)], S[(0<=H)&(H<120)])
    R[(0<=H)&(H<120)] = f2(I[(0<=H)&(H<120)], S[(0<=H)&(H<120)], H[(0<=H)&(H<120)])
    G[(0<=H)&(H<120)] = f3(I[(0<=H)&(H<120)], R[(0<=H)&(H<120)], B[(0<=H)&(H<120)])

    # for 120 <= H < 240
    H = np.subtract(H,120)
    R[(0<=H)&(H<120)] = f1(I[(0<=H)&(H<120)], S[(0<=H)&(H<120)])
    G[(0<=H)&(H<120)] = f2(I[(0<=H)&(H<120)], S[(0<=H)&(H<120)], H[(0<=H)&(H<120)])
    B[(0<=H)&(H<120)] = f3(I[(0<=H)&(H<120)], R[(0<=H)&(H<120)], G[(0<=H)&(H<120)])

    # for 240 <= H < 360
    H = np.subtract(H,120)
    G[(0<=H)&(H<120)] = f1(I[(0<=H)&(H<120)], S[(0<=H)&(H<120)])
    B[(0<=H)&(H<120)] = f2(I[(0<=H)&(H<120)], S[(0<=H)&(H<120)], H[(0<=H)&(H<120)])
    R[(0<=H)&(H<120)] = f3(I[(0<=H)&(H<120)], G[(0<=H)&(H<120)], B[(0<=H)&(H<120)])

    # # Round
    # RGB = np.round(np.dstack( ((zmax*R) , (zmax*G) , (zmax*B)) ))
    
    # # Clip to [0, 255] and convert to uint8
    # RGB = np.clip(RGB, 0, 255).astype(np.uint8)
    
    RGB = np.dstack( (R, G, B) )
    RGB = np.clip(RGB, 0, 1)

    return RGB

# View custom contrast functions applied on images, using their extreme values

In [ ]:
# For an easier plotting, create lambda functions for each function with a value assigned
f1 = lambda x: gamma(x, gamma_range[0])
f1.__name__ = 'gamma_+' + str(gamma_range[0])

f2 = lambda x: gamma(x, gamma_range[1])
f2.__name__ = 'gamma_+' + str(gamma_range[1])

f3 = lambda x: gamma(x, -gamma_range[1])
f3.__name__ = 'gamma_-' + str(gamma_range[1])

f4 = lambda x: gamma(x, -gamma_range[0])
f4.__name__ = 'gamma_-' + str(gamma_range[0])

f5 = lambda x: sigmoid_limits_simmetric(x, sigmoid_range[0])
f5.__name__ = 'sigmoid_+' + str(sigmoid_range[0])

f6 = lambda x: sigmoid_limits_simmetric(x, sigmoid_range[1])
f6.__name__ = 'sigmoid_+' + str(sigmoid_range[1])

f7 = lambda x: sigmoid_limits_simmetric(x, -sigmoid_range[1])
f7.__name__ = 'sigmoid_-' + str(sigmoid_range[1])

f8 = lambda x: sigmoid_limits_simmetric(x, -sigmoid_range[0])
f8.__name__ = 'sigmoid_-' + str(sigmoid_range[0])

functions = [f1, f2, f3, f4, f5, f6, f7, f8]

In [ ]:
# Function to apply transformations
def apply_fixed_transformation(image: np.ndarray, channels, function):
    '''
    Applies specified function to given image. Image must be np.float, normalized to [0, 1]
    '''
    new_image = np.copy(image)
    for ch in channels:
        new_image[:,:,ch] = function(image[:,:,ch])
    return new_image

def plot_histogram_per_channel(image, channels, channel_names):
    colors = ['r', 'g', 'b']

    for i, ch in enumerate(channels):
        # Get image channel
        channel = image[:,:,ch]

        # Remove pixels with '0' value --> Because there are so many 0 pixels that histogram will not be visible
        channel_without_0 = channel.ravel()
        channel_without_0 = channel_without_0[np.where(channel_without_0 > 0)]

        # Plot channel histogram
        h, b = np.histogram(channel_without_0, bins=255, range=(1, 255))
        plt.plot(b[:-1], h, color=colors[i], label=channel_names[i])

    plt.legend(loc='best')

# This function permits the user see how these contrasts functions work on the images
def view_contrast_transformations(images, functions, channels, color_mode):
    num_rows = len(functions)
    num_cols = 4

    for image in images:
        # Get HSI image
        image_hsi = rgb_to_hsi(image)

        plt.subplots(num_rows, num_cols, figsize=(28,28))

        i = 1
        for f in functions:
            plt.subplot(num_rows, num_cols, i)
            plt.imshow(image)
            plt.xticks([]); plt.yticks([])
            plt.title('Original RGB')
            i += 1

            plt.subplot(num_rows, num_cols, i)
            plot_histogram_per_channel(image, [0, 1, 2], 'RGB')
            plt.title('Histogram of original RGB')
            i += 1

            if color_mode == 'RGB':
                # Normalize RGB 
                prepared_image = image.astype(np.float) / 255.0
            else:
                # Get HSI normalized image
                prepared_image = rgb_to_hsi(image)

            # Apply transformation to specified channels
            transformed = apply_fixed_transformation(prepared_image, channels, function=f)

            # Convert it to [0, 255] RGB

            if color_mode == 'RGB':
                # Normalize RGB 
                transformed = np.round(transformed * 255.0).astype(np.uint8)
            else:
                # Get HSI normalized image
                transformed = np.round(hsi_to_rgb(transformed) * 255.0).astype(np.uint8)

            plt.subplot(num_rows, num_cols, i)
            plot_histogram_per_channel(transformed, [0, 1, 2], 'RGB')
            plt.title('Histogram of modified RGB with ' + f.__name__)
            i += 1

            plt.subplot(num_rows, num_cols, i)
            plt.imshow(transformed)
            plt.title('Transformed RGB image with ' + f.__name__)
            plt.xticks([]); plt.yticks([])
            i += 1

        plt.show()

# Testing contrast functions on different color spaces and color channels

In [ ]:
channels=[0,1] # Possible values: [0, 1, 2]
color_mode='RGB' # Possible values: 'HSI' or 'RGB'

# Contrast transformations on normal images
view_contrast_transformations(normal_images, functions, channels, color_mode)

# Contrast transformations on limit images
view_contrast_transformations(limit_images, functions, channels, color_mode)

# Define data augmenter

In [ ]:
# Some helpful functions
def middle(range):
    '''
    Receives a tuple of 2 numbers (range) and returns their middle point
    '''
    return (range[0] + range[1]) / 2

def get_reduced_range(range, *percentages):
    '''
    Receives a tuple of 2 numbers (range) and 1 or 2 parameters.
    If it is 1, it will return a range reduced by the percentage given.
    If it is 2 arguments, it will return a range reduced by the percentage given for each side.
    '''
    length_range = np.abs(range[0] - range[1])
    mid = middle(range)
    mid_length = length_range / 2
    if len(percentages) == 1:
        return mid - mid_length * percentages[0], mid + mid_length * percentages[0]
    elif len(percentages) == 2:
        return mid - mid_length * percentages[0], mid + mid_length * percentages[1]
    else:
        raise Exception('Incorrect number of arguments in \'percentages\'. It must be 1 or 2 arguments')

# This function permits call different contrast functions with different ranges
def apply_contrast_transformation(images, random_state, function, range_values, sign):
    '''
    Receives a batch of images as a list
    A np.random.RandomState object to perform random operations
    A function to perform contrast changes
    A range of values. It must be specified as (positive min, positive max)
    An integer 'sign' whose value will make the function uses full symmetric range or not
    - 0: only positive range
    - 1: only negative range
    - 2: full symmetric range
    Returns a batch of transformed images
    '''

    # Get random value a function parameter
    # Choose range
    if sign == 0:
        # Positive values --> Keeps range
        choosen_range = range_values
    elif sign == 1:
        # Negative values --> Inverted range and multiplied by -1
        choosen_range = (-range_values[1], -range_values[0])
    else:
        # Positive range if random value is lower than 0.5, else negative range
        choosen_range = range_values if random_state.rand(1)[0] < 0.5 else (-range_values[1], -range_values[0])
    
    def get_random_value_in_range(rng):
        # Rng: (min, max)
        
        # (Max - Min) * U(0, 1) + Min --> U(Min, Max)
        return (rng[1] - rng[0]) * random_state.rand(1)[0] + rng[0]

    random_argument_value = get_random_value_in_range(choosen_range)

    # Choose color channels where function will work
    # Randomly choose between HSI (I, S, SI) or RGB (RGB, RG) # Removed RG
    # HSI channels will have '-1' as identifier
    # RGB, '-2'
    hsi_channels = [[-1] + x for x in [[2], [1], [1,2]] ]
    rgb_channels = [[-2] + x for x in [[0,1,2] ]]
    color_channels = hsi_channels + rgb_channels
    # Get channels
    color_channel = color_channels[random_state.randint(0, len(color_channels))]

    # Transformed images
    transformed_batch = []

    for i in range(len(images)):
        current_image = images[i]

        if color_channel[0] == -1:
            # Get HSI normalized image
            prepared_image = rgb_to_hsi(current_image)
        else:
            # Normalize RGB 
            prepared_image = current_image.astype(np.float) / 255.0

        # Apply contrast function on each channel
        transformed_img = np.copy(prepared_image)
        for ch in color_channel[1:]:
            transformed_img[:,:,ch] = function(prepared_image[:,:,ch], random_argument_value)

        # Convert it back to [0,255] RGB image
        if color_channel[0] == -1:
            transformed_img = np.round(hsi_to_rgb(transformed_img) * 255).astype(np.uint8)
        else:
            transformed_img = np.round(transformed_img * 255).astype(np.uint8)

        # Copy image to new batch
        transformed_batch.append(transformed_img)

    return transformed_batch

In [ ]:
# These are the augmenter objects which will be used to perform data-augmentation

# Noise
noise_aug = iaa.OneOf([
    iaa.AddElementwise(uniform_noise_range, per_channel=True), # Uniform noise
    iaa.AdditiveGaussianNoise(scale=(0, gaussian_noise_desv), per_channel=True),
    iaa.AdditiveLaplaceNoise(scale=(0, laplacian_noise), per_channel=True),
    iaa.AdditivePoissonNoise((0, poisson_noise), per_channel=True)
])

# Avoid apply full brightness range when using gamma contrast function
# Gamma with positive values increases contrast on lighter pixels and decreases it on darker ones. Incompatible with decreasing brightness
# Gamma with negative values has the opposite effect, increases contrast on darker pixels. Incompatible with increasing brightness

gamma_aug = iaa.Sequential([
    noise_aug,
    # Al 75% de las imagenes se les aplica alguna de las simetrías
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5),
    iaa.Sometimes(0.5, iaa.Affine(rotate=rotation_range)), # 50%
    iaa.Sometimes(0.5, iaa.AddToHue(value=hue_offset_range)), # 50%
    iaa.OneOf([
        iaa.Sequential([
            # Gamma using only positive values
            iaa.Lambda(func_images=lambda images, random_state, parents, hooks: apply_contrast_transformation(images, random_state, gamma, gamma_range, sign=0)),
            # Brightness range reduced on its negative values
            iaa.AddToBrightness(add=get_reduced_range(brightness_range, 0.20, 1), to_colorspace='YCrCb')
        ], random_order=True),

        iaa.Sequential([
            # Gamma using only negative values
            iaa.Lambda(func_images=lambda images, random_state, parents, hooks: apply_contrast_transformation(images, random_state, gamma, gamma_range, sign=1)),
            # Brightness range reduced on its positive values
            iaa.AddToBrightness(add=get_reduced_range(brightness_range, 1, 0.3), to_colorspace='YCrCb')
        ], random_order=True),
    ])
], random_order=True)

sigmoid_aug = iaa.Sequential([
    noise_aug,
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5),
    iaa.Sometimes(0.5, iaa.Affine(rotate=rotation_range)), # 50%
    iaa.Sometimes(0.5, iaa.AddToHue(value=hue_offset_range)), # 50%
    iaa.Lambda(func_images=lambda images, random_state, parents, hooks: apply_contrast_transformation(images, random_state, sigmoid_limits_simmetric, sigmoid_range, sign=2)),
    iaa.AddToBrightness(add=brightness_range, to_colorspace='YCrCb'),
], random_order=True)

augmenter = iaa.OneOf([
    sigmoid_aug,
    gamma_aug
])

def py_apply_data_augmentation(image: np.ndarray):
    return augmenter.augment_image(image)

# Data augmentation applied randomly on images of both datasets

In [ ]:
# This function permits the user see how data augmentation is randomly applied on every image
def view_data_augmentation(dataset, augmenter, n_times=4):
    
    plt.subplots(n_times + 1, len(dataset), figsize=(22,22))

    count = 1
    # Show original images at first row
    for image in dataset:
        plt.subplot(n_times + 1, len(dataset), count)
        plt.imshow(image)
        plt.title('Original')
        plt.xticks([]); plt.yticks([])
        count += 1
    
    # Apply data augmentations
    for i in range(n_times):
        for image in dataset:
            # Get image
            augmented_image = augmenter.augment_image(image)

            plt.subplot(n_times + 1, len(dataset), count)
            plt.imshow(augmented_image)
            plt.title('Augmented ' + str(i+1))
            plt.xticks([]); plt.yticks([])

            count += 1

    plt.show()

In [ ]:
# View data augmentation on 'normal' images
view_data_augmentation(normal_images, augmenter)

# View data augmentation on 'limit' images
view_data_augmentation(limit_images, augmenter)